# OMERO Micro-SAM Annotation Workflow

This notebook demonstrates the new streamlined workflow using the `omero-annotate-ai` package focused on micro-SAM.

### Instructions:
- **Connection**: Use the secure connection widget below - no more manual password entry!
- **Processing**: This notebook supports processing images from various OMERO object types: images, datasets, projects, plates, and screens.
- **Configuration**: Specify the container type and ID using the interactive widget below.

## 1. Install and Import the Package

**Note**: If you haven't installed the package yet, run:
```bash
pip install -e .
```

For OMERO functionality, also install:
```bash
pip install -e .[omero]
```

In [3]:
# Import the new package
import omero_annotate_ai
from omero_annotate_ai import create_config_widget, create_pipeline, create_default_config, create_omero_connection_widget

# OMERO-related imports
import omero
from omero.gateway import BlitzGateway
try:
    import ezomero
    OMERO_AVAILABLE = True
except ImportError:
    print("⚠️  ezomero not available. Install with: pip install -e .[omero]")
    OMERO_AVAILABLE = False

# System and utility imports
import os
import tempfile
import shutil
from dotenv import load_dotenv

print(f"📦 omero-annotate-ai version: {omero_annotate_ai.__version__}")
print(f"🔗 OMERO functionality: {'✅ Available' if OMERO_AVAILABLE else '❌ Not available'}")
print(f"🔐 Secure connection widget: ✅ Available")

# Check keyring availability
try:
    import keyring
    print(f"🔑 Keyring support: ✅ Available")
except ImportError:
    print(f"🔑 Keyring support: ⚠️ Not available (manual password entry only)")

📦 omero-annotate-ai version: 0.1.0
🔗 OMERO functionality: ✅ Available
🔐 Secure connection widget: ✅ Available
🔑 Keyring support: ✅ Available


## 2. Setup Connection with OMERO

The package now provides a secure, user-friendly widget for OMERO connections with the following features:

### 🔐 Security Features:
- **Secure password storage** using OS-native keychain (Windows Credential Manager, macOS Keychain, Linux GNOME Keyring)
- **Password expiration** options from 1 hour to permanent
- **No plain-text passwords** in configuration files

### 🔗 Integration Features:
- **Auto-loading** from existing `.env` and `.ezomero` files
- **Connection testing** before saving credentials
- **Backward compatibility** with existing workflows

### 🎯 User Experience:
- **Interactive widget** with clear visual feedback
- **Show/hide password** toggle
- **One-click connection** with status indicators

In [4]:
# Create OMERO connection using the new secure widget
from omero_annotate_ai import create_omero_connection_widget

print("🔌 OMERO Connection Setup")
print("Use the widget below to connect to your OMERO server securely:")
print("  • Auto-loads from .env and .ezomero files")
print("  • Secure password storage in OS keychain")
print("  • Password expiration options")
print("  • Connection testing before saving")

# Create and display the connection widget
conn_widget = create_omero_connection_widget()
conn_widget.display()

print("\n📝 Instructions:")
print("1. Fill in your OMERO server details (host, username, password)")
print("2. Choose 'Remember password for' option if desired")
print("3. Click 'Test Connection' to verify your credentials")
print("4. Click 'Save & Connect' to establish the connection")
print("5. Continue to the next cell once connected")

🔌 OMERO Connection Setup
Use the widget below to connect to your OMERO server securely:
  • Auto-loads from .env and .ezomero files
  • Secure password storage in OS keychain
  • Password expiration options
  • Connection testing before saving



📝 Instructions:
1. Fill in your OMERO server details (host, username, password)
2. Choose 'Remember password for' option if desired
3. Click 'Test Connection' to verify your credentials
4. Click 'Save & Connect' to establish the connection
5. Continue to the next cell once connected


In [12]:
# Get the OMERO connection from the widget
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("❌ No OMERO connection established. Please use the widget above to connect.")

print("✅ OMERO connection ready!")
print(f"👤 User: {conn.getUser().getName()}")
print(f"🏢 Group: {conn.getGroupFromContext().getName()}")

# Create temporary work directory
import tempfile
import os
output_directory = os.path.normcase(tempfile.mkdtemp())
print(f"📁 Created temporary work directory: {output_directory}")

# Show connection status
print(f"\n🔗 Connection Status:")
print(f"   Host: {conn._BlitzGateway__ic.getProperties().getProperty('omero.host')}")
print(f"   Secure: {conn.isSecure()}")
print(f"   Keep-alive: Enabled")

ConnectionError: ❌ No OMERO connection established. Please use the widget above to connect.

## 3. Interactive Configuration

Use the interactive widget to configure your micro-SAM annotation workflow. The widget provides:
- **OMERO Connection**: Specify container type and ID
- **Micro-SAM Model**: Choose model type (vit_b, vit_l, vit_h, vit_b_lm)
- **Image Processing**: Configure timepoints, z-slices, channels, 3D mode
- **Patches**: Optional patch extraction settings
- **Training**: Data organization and validation
- **Workflow**: Resume options and output settings

In [13]:
# Create the configuration widget
config_widget = create_config_widget()
config_widget.display()

print("📝 Configure your micro-SAM annotation workflow using the widget above:")
print("   1. Set your OMERO container details (Dataset ID, etc.)")
print("   2. Choose your micro-SAM model (vit_b, vit_l, vit_h, vit_b_lm)")
print("   3. Adjust processing parameters as needed")
print("   4. Click 'Update Configuration' to apply changes")
print("   5. Click 'Validate' to check your configuration")
print("   6. Click 'Show YAML' to see the configuration")

📝 Configure your micro-SAM annotation workflow using the widget above:
   1. Set your OMERO container details (Dataset ID, etc.)
   2. Choose your micro-SAM model (vit_b, vit_l, vit_h, vit_b_lm)
   3. Adjust processing parameters as needed
   4. Click 'Update Configuration' to apply changes
   5. Click 'Validate' to check your configuration
   6. Click 'Show YAML' to see the configuration


## 4. Optional: Set Training Set Name

Choose a specific name for your training set. This helps with organization and allows resuming interrupted workflows.

In [6]:
import pandas as pd

# Set a name for the training set
# Use a specific name if you want to resume from an existing table
trainingset_name = "training_data_20240618"  # Fixed name for resuming
# trainingset_name = "training_data_" + pd.Timestamp.now().strftime("%Y%m%d_%H%M")  # Auto-generated name

print(f"🎯 Training Set Name: {trainingset_name}")

# Update the configuration with the training set name
config = config_widget.get_config()
config.training.trainingset_name = trainingset_name
config.batch_processing.output_folder = output_directory

print(f"📁 Output directory: {output_directory}")

🎯 Training Set Name: training_data_20240618


NameError: name 'output_directory' is not defined

## 5. Validate Configuration and Show Preview

Before running the pipeline, let's validate the configuration and preview what will be processed.

In [ ]:
# Update configuration from widget
config = config_widget.get_config()
config.training.trainingset_name = trainingset_name
config.batch_processing.output_folder = output_directory

# Validate configuration
try:
    config.validate()
    print("✅ Configuration is valid!")
except ValueError as e:
    print(f"❌ Configuration validation failed:")
    print(f"   {e}")
    raise

# Show configuration summary
print("\n📋 Configuration Summary:")
print(f"   🔬 Model: {config.microsam.model_type}")
print(f"   📦 Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"   📺 Channel: {config.omero.channel}")
print(f"   🎯 Training Set: {config.training.trainingset_name}")
print(f"   📊 Batch Size: {config.batch_processing.batch_size} (0 = all in one batch)")
print(f"   🧩 Use Patches: {config.patches.use_patches}")
if config.patches.use_patches:
    print(f"   📐 Patch Size: {config.patches.patch_size}")
    print(f"   📈 Patches per Image: {config.patches.patches_per_image} (non-overlapping)")
print(f"   🔄 Resume from Table: {config.workflow.resume_from_table}")
print(f"   📖 Read-only Mode: {config.workflow.read_only_mode}")
print(f"   🧠 3D Processing: {config.microsam.three_d}")

# Create the pipeline and preview container contents
pipeline = create_pipeline(config, conn)

# Get container details
container_type = config.omero.container_type
container_id = config.omero.container_id

# Validate container exists and get details
if container_type == "dataset":
    container = conn.getObject("Dataset", container_id)
    if container is None:
        raise ValueError(f"Dataset with ID {container_id} not found")
    print(f"\n📁 Dataset: {container.getName()} (ID: {container_id})")
    print(f"📝 Description: {container.getDescription() or 'No description'}")
    
elif container_type == "project":
    container = conn.getObject("Project", container_id)
    if container is None:
        raise ValueError(f"Project with ID {container_id} not found")
    print(f"\n📂 Project: {container.getName()} (ID: {container_id})")
    print(f"📝 Description: {container.getDescription() or 'No description'}")
    
elif container_type == "plate":
    container = conn.getObject("Plate", container_id)
    if container is None:
        raise ValueError(f"Plate with ID {container_id} not found")
    print(f"\n🧪 Plate: {container.getName()} (ID: {container_id})")
    print(f"📝 Description: {container.getDescription() or 'No description'}")
    
elif container_type == "image":
    container = conn.getObject("Image", container_id)
    if container is None:
        raise ValueError(f"Image with ID {container_id} not found")
    print(f"\n🖼️ Image: {container.getName()} (ID: {container_id})")
    
else:
    raise ValueError(f"Unsupported container type: {container_type}")

# Get list of images that will be processed
try:
    images_list = pipeline.get_images_from_container()
    print(f"\n📊 Found {len(images_list)} images to process")
    
    # Show first few images
    print("\n🖼️ Sample images:")
    for i, img in enumerate(images_list[:5]):
        if hasattr(img, 'getName'):  # OMERO image object
            print(f"   {i+1}. {img.getName()} (ID: {img.getId()})")
        else:  # Image ID
            img_obj = conn.getObject("Image", img)
            if img_obj:
                print(f"   {i+1}. {img_obj.getName()} (ID: {img})")
    
    if len(images_list) > 5:
        print(f"   ... and {len(images_list) - 5} more images")
        
except Exception as e:
    print(f"❌ Error getting images from container: {e}")
    raise

In [7]:
# Create the pipeline (but don't run it yet)
pipeline = create_pipeline(config, conn)

# Get container details
container_type = config.omero.container_type
container_id = config.omero.container_id

# Validate container exists and get details
if container_type == "dataset":
    container = conn.getObject("Dataset", container_id)
    if container is None:
        raise ValueError(f"Dataset with ID {container_id} not found")
    print(f"📁 Dataset: {container.getName()} (ID: {container_id})")
    print(f"📝 Description: {container.getDescription() or 'No description'}")
    
elif container_type == "project":
    container = conn.getObject("Project", container_id)
    if container is None:
        raise ValueError(f"Project with ID {container_id} not found")
    print(f"📂 Project: {container.getName()} (ID: {container_id})")
    print(f"📝 Description: {container.getDescription() or 'No description'}")
    
elif container_type == "plate":
    container = conn.getObject("Plate", container_id)
    if container is None:
        raise ValueError(f"Plate with ID {container_id} not found")
    print(f"🧪 Plate: {container.getName()} (ID: {container_id})")
    print(f"📝 Description: {container.getDescription() or 'No description'}")
    
elif container_type == "image":
    container = conn.getObject("Image", container_id)
    if container is None:
        raise ValueError(f"Image with ID {container_id} not found")
    print(f"🖼️ Image: {container.getName()} (ID: {container_id})")
    
else:
    raise ValueError(f"Unsupported container type: {container_type}")

# Get list of images that will be processed
try:
    images_list = pipeline.get_images_from_container()
    print(f"\n📊 Found {len(images_list)} images to process")
    
    # Show first few images
    print("\n🖼️ Sample images:")
    for i, img in enumerate(images_list[:5]):
        if hasattr(img, 'getName'):  # OMERO image object
            print(f"   {i+1}. {img.getName()} (ID: {img.getId()})")
        else:  # Image ID
            img_obj = conn.getObject("Image", img)
            if img_obj:
                print(f"   {i+1}. {img_obj.getName()} (ID: {img})")
    
    if len(images_list) > 5:
        print(f"   ... and {len(images_list) - 5} more images")
        
except Exception as e:
    print(f"❌ Error getting images from container: {e}")
    raise

📂 Project: Senescence (ID: 101)
📝 Description: No description
📁 Loading images from project 101
📊 Found 6 images

📊 Found 6 images to process

🖼️ Sample images:
   1. r01c06.tif (ID: 252)
   2. r01c05.tif (ID: 255)
   3. r01c02.tif (ID: 251)
   4. r01c01.tif (ID: 256)
   5. r01c04.tif (ID: 253)
   ... and 1 more images


In [8]:
print("🚀 Starting annotation pipeline...")
print(f"   This will process {len(images_list)} images using micro-SAM")
print(f"   Model: {config.microsam.model_type}")
print(f"   Napari will open for interactive annotation")
print(f"   Close napari windows when annotation is complete for each batch")

try:
    # Run the complete workflow
    table_id, processed_images = pipeline.run_full_workflow()
    
    print(f"\n✅ Annotation pipeline completed successfully!")
    print(f"📊 Processed {len(processed_images)} images")
    print(f"📋 Tracking table ID: {table_id}")
    
    if config.workflow.read_only_mode:
        print(f"💾 Annotations saved locally to: {config.workflow.local_output_dir}")
    else:
        print(f"☁️ Annotations uploaded to OMERO")
        
except Exception as e:
    print(f"❌ Error during annotation pipeline: {e}")
    raise

🚀 Starting annotation pipeline...
   This will process 6 images using micro-SAM
   Model: vit_l
   Napari will open for interactive annotation
   Close napari windows when annotation is complete for each batch
📁 Loading images from project 101
📊 Found 6 images
🚀 Starting micro-SAM annotation pipeline
📊 Processing 6 images with model: vit_l
📋 Creating new tracking table: microsam_training_training_data_20240618


/home/maarten/miniconda3/envs/micro-sam/lib/python3.12/site-packages/omero_annotate_ai/omero/omero_functions.py:101: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).infer_objects(copy=False).astype(bool)
/home/maarten/miniconda3/envs/micro-sam/lib/python3.12/site-packages/omero_annotate_ai/omero/omero_functions.py:101: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).infer_objects(copy=False).astype(bool)
/home/maarten/miniconda3/envs/micro-sam/lib/python3.12/site-packages/omero_annotate_ai/omero

📋 Created tracking table 'microsam_training_training_data_20240618' with 6 units
   Container: project 101
   Table ID: 670
object group 0
Stored configuration as annotation ID: 671
📋 Getting unprocessed units from table 670
📋 Found 6 unprocessed units
📋 Found 6 processing units
🔄 Processing batch 1/1
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy (required for micro-SAM)...
   Processing chunk 1/1
✅ Successfully loaded 1 images
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy (required for micro-SAM)...
   Processing chunk 1/1
✅ Successfully loaded 1 images
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy (required for micro-SAM)...
   Processing chunk 1/1
✅ Successfully loaded 1 images
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy (required for micro-SAM)...
   Processing chunk 1/1
✅ Successfully loaded 1 images
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy (required for m

Precompute state for files:   0%|          | 0/6 [01:02<?, ?it/s]


KeyboardInterrupt: 

## 6. Run the Annotation Pipeline

Now we'll run the annotation workflow. This will:
1. Create or resume from a tracking table
2. Process images in batches (or all at once if batch_size=0)
3. Launch napari for interactive annotation
4. Upload results to OMERO (or save locally if in read-only mode)

In [ ]:
print("🚀 Starting annotation pipeline...")
print(f"   This will process {len(images_list)} images using micro-SAM")
print(f"   Model: {config.microsam.model_type} (default: vit_b_lm)")
if config.batch_processing.batch_size == 0:
    print(f"   Processing: All images in one batch")
else:
    print(f"   Processing: Batches of {config.batch_processing.batch_size} images")
print(f"   Napari will open for interactive annotation")
print(f"   Close napari windows when annotation is complete for each batch")

try:
    # Run the complete workflow
    table_id, processed_images = pipeline.run_full_workflow()
    
    print(f"\n✅ Annotation pipeline completed successfully!")
    print(f"📊 Processed {len(processed_images)} images")
    print(f"📋 Tracking table ID: {table_id}")
    
    if config.workflow.read_only_mode:
        print(f"💾 Annotations saved locally to: {config.workflow.local_output_dir}")
    else:
        print(f"☁️ Annotations uploaded to OMERO")
        
except Exception as e:
    print(f"❌ Error during annotation pipeline: {e}")
    raise

## 7. Display Results

Let's examine the tracking table and results.

In [ ]:
# Save configuration to file
config_filename = f"annotation_config_{trainingset_name}.yaml"
config.save_yaml(config_filename)
print(f"💾 Configuration saved to: {config_filename}")

# Display the YAML configuration
print("\n📄 YAML Configuration:")
print("=" * 50)
print(config.to_yaml())

# Show how to load it back
print("\n🔄 To reuse this configuration:")
print(f"```python")
print(f"from omero_annotate_ai import load_config")
print(f"config = load_config('{config_filename}')")
print(f"```")

## 8. Configuration Export/Import

Save your configuration for future use or sharing.

In [ ]:
# Get micro-SAM specific parameters
microsam_params = config.get_microsam_params()
print(f"🔧 Micro-SAM parameters:")
for key, value in microsam_params.items():
    print(f"   {key}: {value}")

# Show configuration for different models
print("\n🤖 Available micro-SAM models:")
models = ["vit_b", "vit_l", "vit_h", "vit_b_lm"]
for model in models:
    print(f"   • {model}")

print(f"\n🔬 Current model: {config.microsam.model_type}")
print(f"🧠 3D processing: {config.microsam.three_d}")
print(f"⏰ Timepoint mode: {config.microsam.timepoint_mode}")
print(f"🔢 Z-slice mode: {config.microsam.z_slice_mode}")

## 9. Advanced: Tool-Specific Parameters

The package supports different AI tools with tool-specific parameters.

## 11. Next Steps & Advanced Usage

### 🔌 OMERO Connection Widget
This notebook now uses the new `create_omero_connection_widget()` for secure OMERO connections:

```python
from omero_annotate_ai import create_omero_connection_widget

# Create connection widget
conn_widget = create_omero_connection_widget()
conn_widget.display()

# Get connection after user interaction
conn = conn_widget.get_connection()
```

### 🔐 Keychain Integration
- Passwords are securely stored in your OS keychain
- Support for password expiration (1 hour to permanent)
- Automatic loading from `.env` and `.ezomero` files
- Fallback to manual entry if keychain unavailable

### 🛠️ Development Tips
- Use `.env` files for development and testing
- Save frequently used configurations as YAML files
- Test connections before running large batches
- Consider using `read_only_mode` for safe testing

### 📚 Documentation
- Package documentation: See `CLAUDE.md` for development guidelines
- Example scripts: Check `example_test/` folder for usage examples
- API reference: Use `help(function_name)` for detailed documentation

In [ ]:
# Clean up temporary directory
try:
    shutil.rmtree(output_directory)
    print(f"🗑️ Removed temporary directory: {output_directory}")
except Exception as e:
    print(f"⚠️ Error removing temporary directory: {e}")

# Close OMERO connection
if 'conn' in locals() and conn is not None:
    conn.close()
    print("🔌 OMERO connection closed")

print("\n✨ Workflow completed successfully!")

## 10. Clean Up